In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso,Ridge
from sklearn.metrics import mean_squared_error,r2_score
import joblib 


# Model Training 
In this part we are going to train our model on our selected features using Ridge Regression 


In [4]:
x_train = pd.read_csv('xtrain.csv',index_col=0)
x_test = pd.read_csv('xtest.csv',index_col=0)
y_train = pd.read_csv('ytrain.csv',index_col=0)
y_test = pd.read_csv('ytest.csv',index_col=0)

# checks 
print(x_train.head(1))
print(x_test.head(1))
print(y_train.head(1))
print(y_test.head(1))

   fruity  caramel  peanutyalmondy  nougat  crispedricewafer  hard  bar  \
0     0.0      1.0             0.0     0.0               0.0   0.0  0.0   

   pluribus  sugarpercent  pricepercent  winpercent  
0       0.0      0.416581       0.32914    0.134388  
   fruity  caramel  peanutyalmondy  nougat  crispedricewafer  hard  bar  \
0     1.0      0.0             0.0     0.0               0.0   1.0  0.0   

   pluribus  sugarpercent  pricepercent  winpercent  
0       0.0      0.737973      0.024109    0.260468  
   chocolate
0          0
   chocolate
0          0


Then we select our 'selected_features' to train the Model only on them.
So we keep only those features from all the training set 

In [13]:
features = pd.read_csv('selected_features.csv')['0'].to_list()
features


['fruity', 'nougat', 'bar', 'winpercent']

In [15]:
x_train = x_train[features]
x_test = x_test[features]
x_test.head(1)

,fruity,nougat,bar,winpercent
0,1.0,0.0,0.0,0.260468


### <u>Train the Model and remember to set the random_state (seed) to reproduce the model. We also use the same alpha value we used while selecting the features  </u>

In [27]:
ridge_model = Ridge(alpha = 0.09,random_state=0)
ridge_model.fit(x_train,y_train)


Ridge(alpha=0.09, random_state=0)

Then we will make predictions with our model first on the training set and then on the test set .

### TRAINING SET 

<ul><li> Mean Squared Error </li></ul>

In [28]:
y_pred = ridge_model.predict(x_train)

# calculate error 
print (f"Train MSE :{mean_squared_error(y_pred,y_train)}\n" )

Train MSE :0.06734871693392859



<ul><li> R2 score </li></ul>

In [29]:
# calculate regression score 
print (f"Train R2 :{r2_score(y_pred,y_train)}\n" )

Train R2 :0.6190794396098427



### TEST SET 

<ul><li> Mean Squared Error </li></ul>

In [30]:
y_pred = ridge_model.predict(x_test)

# calculate error 
print (f"Test MSE :{mean_squared_error(y_pred,y_test)}\n" )

Test MSE :0.07588501684067529



<ul><li> R2 score </li></ul>

In [31]:
# calculate regression score 
print (f"Test R2 :{r2_score(y_pred,y_test)}\n" )

Test R2 :0.5878699367181376

